In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime, timedelta
from TFM_PredCurve_Tools import data_date_hour_info, convert_to_df_curve

In [134]:
#pd.reset_option('precision')
pd.options.display.float_format = '{:,.2f}'.format

In [135]:
unit = 'PALOS1'

In [136]:
#Storing df locally
df_tot_lagT = pd.read_csv('/home/dsc/Repos/TFM/df_tot_lagT_PALOS1.csv',index_col=0)

In [137]:
df_set_dE = df_tot_lagT[['Year','Month','Day','Period','Date','Weekday','Block',
                      'Energy','Price','Energy_24h','Energy_168h','Price_24h','Price_168h', 'Marg_Price_24h']]

In [138]:
df_set_dE['Date'] = pd.to_datetime(df_set_dE['Date'])

<ipython-input-138-945412c19bbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_set_dE['Date'] = pd.to_datetime(df_set_dE['Date'])


In [139]:
block_max = df_tot_lagT['Block'].max()

In [140]:
df_set_dE.dtypes

Year                       int64
Month                      int64
Day                        int64
Period                     int64
Date              datetime64[ns]
Weekday                   object
Block                      int64
Energy                   float64
Price                    float64
Energy_24h               float64
Energy_168h              float64
Price_24h                float64
Price_168h               float64
Marg_Price_24h           float64
dtype: object

#### TEST SET

In [141]:
df_set_dE_TEST = df_set_dE[(df_set_dE['Year'] == df_set_dE['Date'].max().year)&
               (df_set_dE['Month'] == df_set_dE['Date'].max().month)]

df_set_dE_TEST = df_set_dE_TEST.reset_index(drop=True)

df_set_dE_TEST_block = [df_set_dE_TEST[df_set_dE_TEST['Block']==block] for block in range(1,block_max+1)]

df_X_dE_TEST = []
y_dE_TEST = []

for df in df_set_dE_TEST_block:
    df_X_dE_TEST.append(df[['Weekday','Period','Energy_24h','Energy_168h',
                            'Price_24h','Price_168h', 'Marg_Price_24h']])
    y_dE_TEST.append(np.array(df[['Energy','Price']]))

#### VALIDATION SET

In [142]:
df_set_dE_VAL = df_set_dE[(df_set_dE['Year'] == (df_set_dE_TEST['Date'].min() - timedelta(days=1)).year)&
               (df_set_dE['Month'] == (df_set_dE_TEST['Date'].min() - timedelta(days=1)).month)]

df_set_dE_VAL = df_set_dE_VAL.reset_index(drop=True)

df_set_dE_VAL_block = [df_set_dE_VAL[df_set_dE_VAL['Block']==block] for block in range(1,block_max+1)]

df_X_dE_VAL = []
y_dE_VAL = []

for df in df_set_dE_VAL_block:
    df_X_dE_VAL.append(df[['Weekday','Period','Energy_24h','Energy_168h',
                           'Price_24h','Price_168h','Marg_Price_24h']])
    y_dE_VAL.append(np.array(df[['Energy','Price']])) #Note y_test is a np.array

#### TRAIN SET

In [143]:
df_set_dE_TRAIN_start = df_set_dE[((df_set_dE['Date'] == df_set_dE['Date'].min()) 
                                   & (df_set_dE['Period'] == 1)  
                                   & (df_set_dE['Block'] == 1))].index

df_set_dE_TRAIN_end = df_set_dE[(df_set_dE['Date'] == df_set_dE_VAL['Date'].min()- timedelta(days=1)) 
                                & (df_set_dE['Period'] == 24)  & (df_set_dE['Block'] == block_max)].index

df_set_dE_TRAIN = df_set_dE.iloc[df_set_dE_TRAIN_start[0]:df_set_dE_TRAIN_end[0]+1]

df_set_dE_num_days = (df_set_dE['Date'].max()-df_set_dE['Date'].min()).days + 1 
df_set_dE_TEST_num_days = (df_set_dE_TEST['Date'].max()-df_set_dE_TEST['Date'].min()).days + 1
df_set_dE_VAL_num_days = (df_set_dE_VAL['Date'].max()-df_set_dE_VAL['Date'].min()).days + 1
df_set_dE_TRAIN_num_days = df_set_dE_num_days - df_set_dE_TEST_num_days - df_set_dE_VAL_num_days
df_set_dE_TRAIN_num_days

df_set_dE_TRAIN_block = [df_set_dE_TRAIN[df_set_dE_TRAIN['Block']==block] for block in range(1,block_max+1)]

df_X_dE_TRAIN = []
y_dE_TRAIN = []

for df in df_set_dE_TRAIN_block:
    df_X_dE_TRAIN.append(df[['Weekday','Period','Energy_24h','Energy_168h',
                             'Price_24h','Price_168h','Marg_Price_24h']])
    y_dE_TRAIN.append(np.array(df[['Energy','Price']])) #Note y_train is a np.array

#### TRAIN + VALIDATION SET

In [144]:
df_X_dE_TRAIN_VAL = []

for block in range(0,len(df_X_dE_TRAIN)):
    df_X_dE_TRAIN_VAL.append(pd.concat([df_X_dE_TRAIN[block], df_X_dE_VAL[block]],axis=0))

In [145]:
y_dE_TRAIN_VAL = []

for block in range(0,len(df_X_dE_TRAIN)):
    y_dE_TRAIN_VAL.append(np.concatenate((y_dE_TRAIN[block], y_dE_VAL[block]),axis=0))

### 3.5.2. NORMALIZATION AND ONE-HOT-ENCODING FOR FEATURES MODELS' INPUT - DELTA ENERGY APPROACH

In this section X_train, X_val, and X_test will be transformed to normalize the Price and Energy features (24h and 168h lagged) and Weekday, and Block (categorical variables) will be transformed using One-Hot-Encoding.

#### TRAIN SET - ENCODING

In [146]:
df_X_dE_TRAIN[0]

,Weekday,Period,Energy_24h,Energy_168h,Price_24h,Price_168h,Marg_Price_24h
13824,Thursday,1,0.00,0.00,180.30,180.30,37.01
13836,Thursday,2,0.00,0.00,180.30,180.30,32.69
13848,Thursday,3,50.00,50.00,1.13,1.13,29.67
13860,Thursday,4,60.00,60.00,1.13,1.13,26.50
13872,Thursday,5,60.00,60.00,1.13,1.13,26.50
...,...,...,...,...,...,...,...
94692,Thursday,20,190.00,190.00,1.13,1.13,50.23
94704,Thursday,21,190.00,190.00,1.13,1.13,52.99
94716,Thursday,22,190.00,190.00,1.13,1.13,50.90
94728,Thursday,23,190.00,190.00,1.13,1.13,45.00


In [147]:
#One-hot encoding will be used to convert variables from categorical to binary.

from sklearn.preprocessing import OneHotEncoder

df_X_TRAIN_cat = []

for df in df_X_dE_TRAIN:
    df_X_TRAIN_cat.append(df[['Weekday','Period']])

one_encoder = OneHotEncoder(drop=None, sparse=False) #Changing 'first' with 'None'

df_X_TRAIN_cat_onehot_encoder = []

for hour in range(0,len(df_X_dE_TRAIN)):
    
    df_X_TRAIN_cat_onehot_encoder.append(one_encoder.fit(df_X_TRAIN_cat[hour]))
    
df_X_TRAIN_cat_onehot = []

for hour in range(0,len(df_X_dE_TRAIN)):
    
    df_X_TRAIN_cat_onehot.append(df_X_TRAIN_cat_onehot_encoder[hour].transform(df_X_TRAIN_cat[hour]))

In [148]:
#Normalization of Energy_tot (24h and 168h lagged) to [0,1], by dividing the entire columns with Pmax.

Energy_tot_max = df_tot_lagT['Pot_max'].max()

df_X_dE_TRAIN_energy_norm = []

for df in df_X_dE_TRAIN:
    df_X_dE_TRAIN_energy_norm.append(df[['Energy_24h', 'Energy_168h']]/Energy_tot_max)

In [149]:
#Normalization of Price (24h and 168h lagged) by stardard method (mean = 0 and std = 1)

from sklearn.preprocessing import StandardScaler

df_X_TRAIN_price = []

for df in df_X_dE_TRAIN:
    df_X_TRAIN_price.append(df[['Price_24h','Price_168h','Marg_Price_24h']])

std_encoder = StandardScaler()

df_X_TRAIN_price_std_encoder = []

for hour in range(0,len(df_X_dE_TRAIN)):
    
    df_X_TRAIN_price_std_encoder.append(std_encoder.fit(df_X_TRAIN_price[hour]))
    

    df_X_TRAIN_price_std = []

for hour in range(0,len(df_X_dE_TRAIN)):
    
    df_X_TRAIN_price_std.append(df_X_TRAIN_price_std_encoder[hour].transform(df_X_TRAIN_price[hour]))

In [150]:
X_dE_TRAIN = []

for hour in range(0,len(df_X_dE_TRAIN)):

    X_dE_TRAIN.append(np.concatenate((df_X_TRAIN_cat_onehot[hour], 
                             df_X_dE_TRAIN_energy_norm[hour],
                             df_X_TRAIN_price_std[hour]),
                            axis=1))

In [151]:
df_X_TRAIN_cat_onehot[0].shape, df_X_dE_TRAIN_energy_norm[0].shape, df_X_TRAIN_price_std[0].shape

((6744, 31), (6744, 2), (6744, 3))

In [152]:
len(df_X_TRAIN_cat_onehot), len(df_X_dE_TRAIN_energy_norm), len(df_X_TRAIN_price_std)

(12, 12, 12)

In [153]:
len(df_X_dE_TRAIN)

12

In [154]:
X_dE_TRAIN[0].shape

(6744, 36)

In [155]:
type(X_dE_TRAIN[0])

numpy.ndarray

#### VALIDATION SET - ENCODING --> FITTING WITH TRAIN SET

In [156]:
#One-hot encoding will be used to convert variables from categorical to binary.

df_X_VAL_cat = []
for df in df_X_dE_VAL:
    df_X_VAL_cat.append(df[['Weekday','Period']])
    
df_X_VAL_cat_onehot = []
for hour in range(0,len(df_X_dE_VAL)):
    df_X_VAL_cat_onehot.append(df_X_TRAIN_cat_onehot_encoder[hour].transform(df_X_VAL_cat[hour]))
    
#Normalization of Energy_tot (24h and 168h lagged) to [0,1], by dividing the entire columns with Pmax.
df_X_dE_VAL_energy_norm = []
for df in df_X_dE_VAL:
    df_X_dE_VAL_energy_norm.append(df[['Energy_24h', 'Energy_168h']]/Energy_tot_max)
    
#Normalization of Price (24h and 168h lagged) by stardard method (mean = 0 and std = 1)
df_X_VAL_price = []

for df in df_X_dE_VAL:
    df_X_VAL_price.append(df[['Price_24h','Price_168h','Marg_Price_24h']])
    
df_X_VAL_price_std = []
for hour in range(0,len(df_X_dE_VAL)):
    df_X_VAL_price_std.append(df_X_TRAIN_price_std_encoder[hour].transform(df_X_VAL_price[hour]))
    
X_dE_VAL = []
for hour in range(0,len(df_X_dE_VAL)):
    X_dE_VAL.append(np.concatenate((df_X_VAL_cat_onehot[hour], 
                             df_X_dE_VAL_energy_norm[hour],
                             df_X_VAL_price_std[hour]),
                            axis=1))

#### TEST SET - ENCODING --> FITTING WITH TRAIN_VAL SET

In [157]:
#One-hot encoding will be used to convert variables from categorical to binary.

from sklearn.preprocessing import OneHotEncoder

one_encoder = OneHotEncoder(drop=None, sparse=False) #Changing 'first' with 'None'

df_X_TEST_cat = []

for df in df_X_dE_TEST:
    df_X_TEST_cat.append(df[['Weekday','Period']])

df_X_TRAIN_VAL_cat = []

for df in df_X_dE_TRAIN_VAL:
    df_X_TRAIN_VAL_cat.append(df[['Weekday','Period']])

df_X_TRAIN_VAL_cat_onehot_encoder = []
for hour in range(0,len(df_X_dE_TRAIN_VAL)):
    df_X_TRAIN_VAL_cat_onehot_encoder.append(one_encoder.fit(df_X_TRAIN_VAL_cat[hour]))

df_X_TRAIN_VAL_cat_onehot = []
for hour in range(0,len(df_X_dE_TRAIN_VAL)):
    df_X_TRAIN_VAL_cat_onehot.append(df_X_TRAIN_VAL_cat_onehot_encoder[hour].transform(df_X_TRAIN_VAL_cat[hour]))
    
df_X_TEST_cat_onehot = []
for hour in range(0,len(df_X_dE_TEST)):
    df_X_TEST_cat_onehot.append(df_X_TRAIN_VAL_cat_onehot_encoder[hour].transform(df_X_TEST_cat[hour]))

In [158]:
#Normalization of Energy_tot (24h and 168h lagged) to [0,1], by dividing the entire columns with Pmax.
df_X_dE_TEST_energy_norm = []
for df in df_X_dE_TEST:
    df_X_dE_TEST_energy_norm.append(df[['Energy_24h', 'Energy_168h']]/Energy_tot_max)
    
df_X_dE_TRAIN_VAL_energy_norm = []
for df in df_X_dE_TRAIN_VAL:
    df_X_dE_TRAIN_VAL_energy_norm.append(df[['Energy_24h', 'Energy_168h']]/Energy_tot_max)

In [159]:
#Normalization of Price (24h and 168h lagged) by stardard method (mean = 0 and std = 1)

from sklearn.preprocessing import StandardScaler

df_X_TEST_price = []

for df in df_X_dE_TEST:
    df_X_TEST_price.append(df[['Price_24h','Price_168h','Marg_Price_24h']])

df_X_TRAIN_VAL_price = []

for df in df_X_dE_TRAIN_VAL:
    df_X_TRAIN_VAL_price.append(df[['Price_24h','Price_168h','Marg_Price_24h']])
    
std_encoder = StandardScaler()

df_X_TRAIN_VAL_price_std_encoder = []

for hour in range(0,len(df_X_dE_TRAIN_VAL)):  
    df_X_TRAIN_VAL_price_std_encoder.append(std_encoder.fit(df_X_TRAIN_VAL_price[hour]))
    
df_X_TRAIN_VAL_price_std = []
for hour in range(0,len(df_X_dE_TRAIN_VAL)):   
    df_X_TRAIN_VAL_price_std.append(df_X_TRAIN_VAL_price_std_encoder[hour].transform(df_X_TRAIN_VAL_price[hour]))
    
df_X_TEST_price_std = []
for hour in range(0,len(df_X_dE_TEST)):   
    df_X_TEST_price_std.append(df_X_TRAIN_VAL_price_std_encoder[hour].transform(df_X_TEST_price[hour]))

In [160]:
X_dE_TEST = []
for hour in range(0,len(df_X_dE_TEST)):
    X_dE_TEST.append(np.concatenate((df_X_TEST_cat_onehot[hour], 
                             df_X_dE_TEST_energy_norm[hour],
                             df_X_TEST_price_std[hour]),
                            axis=1))

In [161]:
X_dE_TRAIN_VAL = []
for hour in range(0,len(df_X_dE_TRAIN_VAL)):
    X_dE_TRAIN_VAL.append(np.concatenate((df_X_TRAIN_VAL_cat_onehot[hour], 
                             df_X_dE_TRAIN_VAL_energy_norm[hour],
                             df_X_TRAIN_VAL_price_std[hour]),
                            axis=1))

## 4. MULTI-OUTPUT REGRESSION MODELS

In this section, several multi-output regression models will be develop to predict Energy_tot and Price.

Test set will be used to adjust the models' hyperparameters and Validation test will be used to calibrate the quality of the models.

In [162]:
type(X_dE_TRAIN[0]), type(y_dE_TRAIN[0])

(numpy.ndarray, numpy.ndarray)

In [163]:
y_dE_TRAIN[0]

array([[  0.  , 180.3 ],
       [  0.  , 180.3 ],
       [ 50.  ,   1.13],
       ...,
       [190.  ,   1.13],
       [190.  ,   1.13],
       [190.  ,   1.13]])

### 4.1. MULTI-OUTPUT RANDOM FOREST REGRESSION - DELTA ENERGY APPROACH


Models are fitted with the TRAIN SET and predictions to fit the hyperparameters are checked with VALIDATION SET.
After selecting the hyperparameters, fit is done with TRAIN + VALIDATION SET and predictions with TEST SET.

In [164]:
import pickle

hours = len(X_dE_TEST)

In [165]:
# RANDOM FOREST
from sklearn.ensemble import RandomForestRegressor

reg_rf = RandomForestRegressor(n_estimators=100)

#### 4.1.1. RANDOM FOREST - BASIC APPROACH

In [166]:
pickle_path = '/home/dsc/Documents/TFM/Data/TFM_Pickle/'

In [167]:
reg_rf_fit_dE_TRAIN_VAL_pkl_names = []

for hour in range(0,hours):    
    model_name = pickle_path + unit + '_rf_fit_dE_TRAIN_VAL_hour_' + str(hour+1) + '.pkl'
    reg_rf_fit_dE_TRAIN_VAL_pkl_names.append(model_name)
    
    #Lines commented because models have been stored already
    #reg_rf_fit_dE_TRAIN_VAL = reg_rf.fit(X_dE_TRAIN_VAL[hour],y_dE_TRAIN_VAL[hour]) 
    #pickle.dump(reg_rf_fit_dE_TRAIN_VAL, open(model_name, 'wb'))
    

y_dE_TEST_pred_rf = []
for hour in range(0,hours):
    loaded_model = pickle.load(open(reg_rf_fit_dE_TRAIN_VAL_pkl_names[hour], 'rb'))
    y_dE_TEST_pred_rf.append(loaded_model.predict(X_dE_TEST[hour]))

In [168]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# MAE
mae_dE_TEST_rf = []
for hour in range(0,hours):
    mae_dE_TEST_rf.append(mean_absolute_error(y_dE_TEST[hour],y_dE_TEST_pred_rf[hour]))
mae_dE_TEST_rf

[0.490584677419383,
 0.3949516801080892,
 0.4068747311833681,
 0.38620409946292855,
 0.40324684139842787,
 0.4001116935490713,
 0.4667708333340622,
 0.36656653225876124,
 0.45599045698997276,
 0.38407587365663376,
 0.4437214381727098,
 0.4834005376353906]

In [169]:
# RMSE from random forest

rmse_dE_TEST_rf = []
for hour in range(0,hours):
    rmse_dE_TEST_rf.append(np.sqrt(mean_squared_error(y_dE_TEST[hour],y_dE_TEST_pred_rf[hour])))
rmse_dE_TEST_rf

[7.695524826296812,
 5.067209166530596,
 4.972208181152495,
 5.080423173894016,
 5.124784796290049,
 5.129593846413766,
 4.838288802005537,
 4.990147643593811,
 4.818278043604714,
 4.496791951526679,
 4.789176983147165,
 7.047096509941528]

In [170]:
np.mean(mae_dE_TEST_rf), np.mean(rmse_dE_TEST_rf)

(0.42354161626406656, 5.3374603270330985)

#### 4.1.2. RANDOM FOREST - SELECTING HYPERPARAMENTERS

In [171]:
#Creating a fixed split for GridSearch
#Since the length of all 24 sets are equal, only is necesary to create one "test_fold"

TRAIN_len = len(X_dE_TRAIN[0])
VAL_len = len(X_dE_VAL[0])

test_fold = np.repeat([-1,0], [TRAIN_len, VAL_len])

In [172]:
from sklearn.model_selection import PredefinedSplit

ps = PredefinedSplit(test_fold)

In [173]:
#Checking the number of splits
ps.get_n_splits()

1

In [175]:
from sklearn.model_selection import GridSearchCV

reg_rf_grid = GridSearchCV(RandomForestRegressor(n_estimators=100),
                  param_grid={"max_depth":np.arange(2,11),
                              "min_samples_leaf":np.arange(10,100,15)},
                  scoring="neg_mean_absolute_error",
                  cv=ps)

In [176]:
hours = len(X_dE_TRAIN)

reg_rf_grid_fit_dE_TRAIN_VAL_pkl_names = []

for hour in range(0,hours):    
    model_name = pickle_path + unit + '_rf_grid_fit_dE_TRAIN_VAL_hour_' + str(hour+1) + '.pkl'
    reg_rf_grid_fit_dE_TRAIN_VAL_pkl_names.append(model_name)
    
    #Lines commented because models have been stored already
    reg_rf_grid_fit_dE_TRAIN_VAL = reg_rf_grid.fit(X_dE_TRAIN_VAL[hour],y_dE_TRAIN_VAL[hour]) 
    pickle.dump(reg_rf_grid_fit_dE_TRAIN_VAL, open(model_name, 'wb'))
    

y_dE_TEST_pred_rf_grid = []
for hour in range(0,hours):
    loaded_model = pickle.load(open(reg_rf_grid_fit_dE_TRAIN_VAL_pkl_names[hour], 'rb'))
    y_dE_TEST_pred_rf_grid.append(loaded_model.predict(X_dE_TEST[hour]))

In [177]:
# MAE - RANDOM FOREST - GRID
mae_dE_TEST_rf_grid = []
for hour in range(0,hours):
    mae_dE_TEST_rf_grid.append(mean_absolute_error(y_dE_TEST[hour],y_dE_TEST_pred_rf_grid[hour]))
mae_dE_TEST_rf_grid

[0.5368701615861622,
 0.3943338711933812,
 0.40734053773862827,
 0.3884959713452878,
 0.41162928885162803,
 0.3919442939568322,
 0.44767762257660804,
 0.3781117399095301,
 0.41829464779619996,
 0.38922237512224284,
 0.40265863702324073,
 0.5297432343110763]

In [178]:
# RMSE from random forest

rmse_dE_TEST_rf_grid = []
for hour in range(0,hours):
    rmse_dE_TEST_rf_grid.append(np.sqrt(mean_squared_error(y_dE_TEST[hour],y_dE_TEST_pred_rf_grid[hour])))
rmse_dE_TEST_rf_grid

[7.664803394001741,
 5.153629434077718,
 5.155998074475404,
 5.115547568150641,
 5.150231546431896,
 5.105393513309137,
 4.995162984665915,
 5.011712668457448,
 4.920790906600088,
 4.873512412598912,
 4.848568938280547,
 7.04377557095602]

In [179]:
np.mean(mae_dE_TEST_rf_grid) , np.mean(rmse_dE_TEST_rf_grid)

(0.42469353178423486, 5.4199272510004555)

In [180]:
np.mean(mae_dE_TEST_rf) , np.mean(rmse_dE_TEST_rf)

(0.42354161626406656, 5.3374603270330985)

### 4.2. MULTI-OUTPUT XG-BOOST REGRESSION - DELTA ENERGY APPROACH

Models are fitted with the TEST SET and predictions to fit the hyperparameters are checked with VALIDATION SET.

In [181]:
#XG-BOOST

from xgboost import XGBRegressor
#from sklearn.ensemble import GradientBoostingRegressor

In [182]:
#reg_XGB = GradientBoostingRegressor(max_depth=4, n_estimators=100,learning_rate=0.1)

reg_XGB = XGBRegressor(max_depth=4, n_estimators=100,learning_rate=0.1)

#### 4.2.1. XG-BOOST REGRESSION - BASIC APPROACH

##### DIRECT WRAPPER

In [183]:
from sklearn.multioutput import MultiOutputRegressor

wrapper_XGB = MultiOutputRegressor(reg_XGB)

In [184]:
   
reg_XGB_fit_dE_TRAIN_VAL_pkl_names = []

for hour in range(0,hours):    
    model_name = pickle_path + unit + '_XGB_fit_dE_TRAIN_VAL_hour_' + str(hour+1) + '.pkl'
    reg_XGB_fit_dE_TRAIN_VAL_pkl_names.append(model_name)
    
    #Lines commented because models have been stored already
    #wrapper_XGB_fit_dE_TRAIN_VAL = wrapper_XGB.fit(X_dE_TRAIN_VAL[hour],y_dE_TRAIN_VAL[hour]) 
    #pickle.dump(wrapper_XGB_fit_dE_TRAIN_VAL, open(model_name, 'wb'))
    

y_dE_TEST_pred_XGB = []
for hour in range(0,hours):
    loaded_model = pickle.load(open(reg_XGB_fit_dE_TRAIN_VAL_pkl_names[hour], 'rb'))
    y_dE_TEST_pred_XGB.append(loaded_model.predict(X_dE_TEST[hour]))

In [185]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# MAE - XGBOOST - TEST
mae_dE_TEST_XGB = []
for hour in range(0,hours):
    mae_dE_TEST_XGB.append(mean_absolute_error(y_dE_TEST[hour],y_dE_TEST_pred_XGB[hour]))
mae_dE_TEST_XGB

[0.5448742880415853,
 0.3674480162641072,
 0.4011862338638771,
 0.37071354850646054,
 0.3656687517632408,
 0.3654080426837185,
 0.3623463263354646,
 0.3515735456812137,
 0.35784611361481944,
 0.3792790669731569,
 0.37486656492996784,
 0.5338099602730121]

In [186]:
# RMSE - XGBOOST - TEST

rmse_dE_TEST_XGB = []
for hour in range(0,hours):
    rmse_dE_TEST_XGB.append(np.sqrt(mean_squared_error(y_dE_TEST[hour],y_dE_TEST_pred_XGB[hour])))
rmse_dE_TEST_XGB

[7.756817058805694,
 5.231179895367529,
 5.272794210178759,
 5.263872178336672,
 5.181358568866595,
 5.1322380107337775,
 5.091897785855892,
 5.101771468465419,
 5.071620316890503,
 4.9450529788261415,
 4.820674632306921,
 7.2450975582128105]

In [187]:
np.mean(mae_dE_TEST_XGB), np.mean(rmse_dE_TEST_XGB)

(0.397918371577552, 5.509531221903892)

##### CHAIN WRAPPER

In [188]:
from sklearn.multioutput import RegressorChain

wrapper_chain_XGB = RegressorChain(reg_XGB, order=[0,1])

In [189]:
reg_chain_XGB_fit_dE_TRAIN_VAL_pkl_names = []

for hour in range(0,hours):    
    model_name = pickle_path + unit + '_chain_XGB_fit_dE_TRAIN_VAL_hour_' + str(hour+1) + '.pkl'
    reg_chain_XGB_fit_dE_TRAIN_VAL_pkl_names.append(model_name)
    
    #Lines commented because models have been stored already
    #chain_XGB_fit_dE_TRAIN_VAL = wrapper_chain_XGB.fit(X_dE_TRAIN_VAL[hour],y_dE_TRAIN_VAL[hour]) 
    #pickle.dump(chain_XGB_fit_dE_TRAIN_VAL, open(model_name, 'wb'))
    

y_dE_TEST_pred_chain_XGB = []
for hour in range(0,hours):
    loaded_model = pickle.load(open(reg_chain_XGB_fit_dE_TRAIN_VAL_pkl_names[hour], 'rb'))
    y_dE_TEST_pred_chain_XGB.append(loaded_model.predict(X_dE_TEST[hour]))

In [190]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# MAE - XGBOOST_chain - TEST
mae_dE_TEST_chain_XGB = []
for hour in range(0,hours):
    mae_dE_TEST_chain_XGB.append(mean_absolute_error(y_dE_TEST[hour],y_dE_TEST_pred_chain_XGB[hour]))
mae_dE_TEST_chain_XGB

[0.5383663083797188,
 0.3810800194740315,
 0.389041000453216,
 0.3648493647319031,
 0.36193781171354755,
 0.37972690285053484,
 0.3727124071425668,
 0.3541109989327147,
 0.3698671740662043,
 0.37977362852744107,
 0.3558573284609039,
 0.5338099602730121]

In [191]:
# RMSE - XGBOOST_chain - TEST

rmse_dE_TEST_chain_XGB = []
for hour in range(0,hours):
    rmse_dE_TEST_chain_XGB.append(np.sqrt(mean_squared_error(y_dE_TEST[hour],y_dE_TEST_pred_chain_XGB[hour])))
rmse_dE_TEST_chain_XGB

[7.741022758755459,
 5.280730637343288,
 5.248488816679602,
 5.2253800378322275,
 5.171486844510415,
 5.125353370720004,
 5.102813539769903,
 5.0670746715015715,
 5.028589598385578,
 4.972457607721138,
 4.916849719396347,
 7.2450975582128105]

In [192]:
np.mean(mae_dE_TEST_chain_XGB), np.mean(rmse_dE_TEST_chain_XGB)

(0.3984277420838162, 5.510445430069029)

In [193]:
# Direct wrapper and Chain wrapper seems to be very similar

#### 4.2.2. XG-BOOST REGRESSION - CHAIN MULTI-OUTPUT WRAPPER APPROACH - SELECTING HYPERPARAMETERS

In [194]:
# X tiene que ser X_TEST + X_VAL y el split según X_TEST / X_VAL

In [195]:
from sklearn.model_selection import GridSearchCV

#We are using the same Predefined Split as for Random Forest

reg_XGB_grid = GridSearchCV(XGBRegressor(n_estimators=100),
                  param_grid={"max_depth":np.arange(2,10),
                             "learning_rate":np.arange(1,10)/10},
                  scoring="neg_mean_absolute_error",
                  cv=ps)

In [196]:
wrapper_chain_XGB_grid = RegressorChain(reg_XGB_grid, order=[0,1])

In [197]:
reg_chain_XGB_grid_fit_dE_TRAIN_VAL_pkl_names = []

for hour in range(0,hours):    
    model_name = pickle_path + unit + '_chain_XGB_grid_fit_dE_TRAIN_VAL_hour_' + str(hour+1) + '.pkl'
    reg_chain_XGB_grid_fit_dE_TRAIN_VAL_pkl_names.append(model_name)
    
    #Lines commented because models have been stored already
    chain_XGB_grid_fit_dE_TRAIN_VAL = wrapper_chain_XGB_grid.fit(X_dE_TRAIN_VAL[hour],y_dE_TRAIN_VAL[hour]) 
    pickle.dump(chain_XGB_grid_fit_dE_TRAIN_VAL, open(model_name, 'wb'))
    

y_dE_TEST_pred_chain_XGB_grid = []
for hour in range(0,hours):
    loaded_model = pickle.load(open(reg_chain_XGB_grid_fit_dE_TRAIN_VAL_pkl_names[hour], 'rb'))
    y_dE_TEST_pred_chain_XGB_grid.append(loaded_model.predict(X_dE_TEST[hour]))

In [198]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# MAE - XGBOOST_chain - TEST
mae_dE_TEST_chain_XGB_grid = []
for hour in range(0,hours):
    mae_dE_TEST_chain_XGB_grid.append(mean_absolute_error(y_dE_TEST[hour],y_dE_TEST_pred_chain_XGB_grid[hour]))
mae_dE_TEST_chain_XGB_grid

[0.4932339501052462,
 0.44846476999662255,
 0.45808613904060835,
 0.3665435746280099,
 0.3869767315549558,
 0.41545557536104694,
 0.42843923035648546,
 0.40384690541253326,
 0.37360453421229944,
 0.44088948824232604,
 0.3885555043007415,
 0.5149122171504469]

In [199]:
# RMSE - XGBOOST_chain - TEST

rmse_dE_TEST_chain_XGB_grid = []
for hour in range(0,hours):
    rmse_dE_TEST_chain_XGB_grid.append(np.sqrt(mean_squared_error(y_dE_TEST[hour],y_dE_TEST_pred_chain_XGB_grid[hour])))
rmse_dE_TEST_chain_XGB_grid

[7.612264186316616,
 5.285087137720951,
 5.276073606537065,
 5.220515041743091,
 5.172064658099694,
 5.137871679576182,
 5.111654865392466,
 5.067560410243103,
 5.023340505832293,
 4.994856193574195,
 4.926149681585347,
 7.50988076611754]

In [200]:
np.mean(mae_dE_TEST_chain_XGB_grid), np.mean(rmse_dE_TEST_chain_XGB_grid)

(0.4265840516967769, 5.528109894394878)

In [201]:
np.mean(mae_dE_TEST_chain_XGB), np.mean(rmse_dE_TEST_chain_XGB)

(0.3984277420838162, 5.510445430069029)

### 5. NEW METRIC CALCULATION: PERIOD CURVE AREA ERROR

In this section a curve area calculation per each period will be calculated in order to compare the original curve and the predicted one.

Area curve will have money (€) units and it will give good information about the model precision.

Four different cuve areas will be calculated:

+ Total curve area: A_tot
+ Area when Price is maximum: A_max
+ Area when Price is minimum: A_min
+ Celtral Area: A_cen = A_tot - A_max - A_min

#### NOTE: Calculating extreme areas when blocks are less than 2 is not possible, so it is checked only when using a complete dataframe (all blocks for all periods).

Extreme areas (when Price is maximum or minimum) can introduce a lot of error in the metric that cound be not very relevant for the objective of the project: since it is not very probably that extreme cases are chosen, the real effective zone for the purpose of the project is to predict well the central part of the curve.

#### TOTAL CURVE AREA METRIC

Different metrics based on curve area can be defined:

* Difference of the total curve area between y_dE and y_dE_pred (absolute and relative)
* Total absolute difference between y_dE and y_dE_pred (absolute and relative)

In [202]:
#DIFFERENCE OF THE TOTAL CURVE AREAS & TOTAL ABSOLUTE DIFFERENCE OF y_dE and y_dE_pred

def area_pred_curve_summary(y, y_pred):
    
    y_TEST_area_tot = []
    y_area_tot_diff_TEST_abs = []
    y_area_tot_diff_TEST_rel = []

    y_TEST_area = []
    y_area_diff_TEST_abs = []
    y_area_diff_TEST_rel = []
    
    hours = len(y)
    
    for hour in range(0, hours):

        y_dE_TEST_area = (y[hour][:,0]*y[hour][:,1])
        y_dE_TEST_pred_area = (y_pred[hour][:,0]*y_pred[hour][:,1])
        area_diff_TEST_abs = abs(y_dE_TEST_area - y_dE_TEST_pred_area).sum() #Total Absolute Area Differences

        y_dE_TEST_area_tot = y_dE_TEST_area.sum() #Total area for the whole set period, for each hour 
        y_dE_TEST_pred_area_tot = y_dE_TEST_pred_area.sum()
        area_tot_diff_TEST_abs = (y_dE_TEST_area_tot - y_dE_TEST_pred_area_tot) #Total Differences of Total Areas

        #Relative area difference
        area_diff_TEST_rel = (area_diff_TEST_abs / y_dE_TEST_area_tot) * 100
        area_tot_diff_TEST_rel = (area_tot_diff_TEST_abs / y_dE_TEST_area_tot) * 100

        y_area_diff_TEST_abs.append(area_diff_TEST_abs)
        y_area_diff_TEST_rel.append(area_diff_TEST_rel)

        y_TEST_area_tot.append(y_dE_TEST_area_tot)
        y_area_tot_diff_TEST_abs.append(area_tot_diff_TEST_abs)
        y_area_tot_diff_TEST_rel.append(area_tot_diff_TEST_rel)

    df_y_TEST_area_summary = pd.DataFrame({'Area_tot(€)': y_TEST_area_tot,
                  'Area_diff_abs(€)': y_area_diff_TEST_abs, 
                  'Area_diff_rel(%)': y_area_diff_TEST_rel,
                  'Area_tot_diff_abs(€)': y_area_tot_diff_TEST_abs, 
                  'Area_tot_diff_rel(%)': y_area_tot_diff_TEST_rel})
    
    return df_y_TEST_area_summary

In [203]:
df_y_TEST_area_pred_rf_grid_summary = area_pred_curve_summary(y_dE_TEST, y_dE_TEST_pred_rf_grid)
df_y_TEST_area_pred_rf_grid_summary

,Area_tot(€),Area_diff_abs(€),Area_diff_rel(%),Area_tot_diff_abs(€),Area_tot_diff_rel(%)
0,"126,147.55","2,943.99",2.33,"-2,591.91",-2.05
1,"301,273.15","6,107.62",2.03,"-1,070.33",-0.36
2,"307,771.74","6,455.04",2.10,"-1,827.84",-0.59
3,"314,305.69","6,037.52",1.92,746.94,0.24
4,"317,316.68","6,619.25",2.09,615.75,0.19
5,"313,261.38","5,635.69",1.80,"-1,188.65",-0.38
6,"319,395.49","7,794.76",2.44,483.17,0.15
7,"325,537.25","5,974.71",1.84,"1,353.43",0.42
8,"335,576.26","7,187.40",2.14,38.20,0.01
9,"345,679.02","6,535.66",1.89,"-1,732.37",-0.50


In [204]:
# It can be seen that the bidding area for hours from 10 to 24 is the same, so the bid is the same 
# for 15 hours per day --> Possibility of reducing models???

In [205]:
df_y_TEST_area_pred_rf_summary = area_pred_curve_summary(y_dE_TEST, y_dE_TEST_pred_rf)
df_y_TEST_area_pred_rf_summary

,Area_tot(€),Area_diff_abs(€),Area_diff_rel(%),Area_tot_diff_abs(€),Area_tot_diff_rel(%)
0,"126,147.55",419.96,0.33,9.44,0.01
1,"301,273.15","6,241.03",2.07,-916.29,-0.30
2,"307,771.74","6,715.60",2.18,"-2,014.65",-0.65
3,"314,305.69","5,997.74",1.91,"1,202.46",0.38
4,"317,316.68","6,225.87",1.96,"1,622.10",0.51
5,"313,261.38","6,286.10",2.01,"-1,862.93",-0.59
6,"319,395.49","8,497.16",2.66,861.26,0.27
7,"325,537.25","5,678.23",1.74,"1,604.60",0.49
8,"335,576.26","8,284.47",2.47,556.53,0.17
9,"345,679.02","6,942.08",2.01,"-2,146.59",-0.62


In [206]:
#INCLUDE HERE MORE INFORMATION FROM THE TABLE & GRAPHICS

In [207]:
#List of df with the comparison of real data and prediction for every hour.
#Returning a complete df for all predictions

def df_pred_summary(y, y_pred):

    df_pred_summary = []
    
    hours = len(y)

    for hour in range(0, hours):

        df_pred = pd.DataFrame(np.concatenate([y[hour],y_pred[hour]],axis=1), 
                                       columns= ['dE','Price','dE_pred','Price_pred'])
        df_pred['dE_diff'] = df_pred['dE'] - df_pred['dE_pred']
        df_pred['Price_diff'] = df_pred['Price'] - df_pred['Price_pred']
        df_pred['Area'] = df_pred['dE'] * df_pred['Price']
        df_pred['Area_pred'] = df_pred['dE_pred'] * df_pred['Price_pred']
        df_pred['Area_diff'] = df_pred['Area'] - df_pred['Area_pred']
        df_pred['Area_diff_rel(%)'] = df_pred['Area_diff']/df_pred['Area']*100
        df_pred['Period'] = hour + 1
        
        df_pred_summary.append(df_pred)

    # Creating a df with all the comparisons (to be able to retreive info easily)

    df_y_pred_summary = pd.DataFrame()

    for hour in range(0, len(df_pred_summary)):
        df_y_pred_summary = pd.concat([df_y_pred_summary,df_pred_summary[hour]])
        
    df_y_pred_summary.fillna(0,inplace=True)
    df_y_pred_summary.reset_index(drop=True,inplace=True)
        
    return df_y_pred_summary

In [208]:
df_TEST_pred_rf_summary  = df_pred_summary(y_dE_TEST, y_dE_TEST_pred_rf)

In [209]:
df_TEST_pred_rf_summary[(df_TEST_pred_rf_summary['Period']==10) & (df_TEST_pred_rf_summary['dE_diff']<-2)]

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
7304,0.00,180.30,13.43,75.12,-13.43,105.18,0.00,"1,008.85","-1,008.85",-inf,10


In [210]:
df_TEST_pred_rf_summary.describe()

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
count,"8,928.00","8,928.00","8,928.00","8,928.00","8,928.00","8,928.00","8,928.00","8,928.00","8,928.00","8,928.00","8,928.00"
mean,32.84,91.97,32.84,92.01,0.00,-0.03,"2,403.38","2,403.93",-0.56,-inf,6.50
std,66.57,71.52,66.57,71.50,3.40,6.87,"9,802.31","9,802.63",520.26,nan,3.45
min,0.00,1.13,0.00,1.13,-135.00,-179.17,0.00,0.00,"-24,340.50",-inf,1.00
25%,0.00,37.64,0.00,37.53,0.00,-0.08,0.00,0.00,-1.31,-0.19,3.75
50%,17.00,42.81,17.00,43.14,0.00,0.00,619.99,623.41,0.00,0.00,6.50
75%,17.00,180.30,17.00,180.30,0.00,0.00,706.35,705.08,0.00,0.00,9.25
max,394.10,180.30,394.10,180.30,137.75,179.17,"71,056.23","71,056.23","24,836.33",100.00,12.00


In [211]:
df_TEST_pred_rf_summary['Area_diff'].mean()

-0.5550705831232754

In [212]:
Area_pred_mean_error_TEST = df_TEST_pred_rf_summary['Area_diff'].mean()
Area_pred_mean_error_TEST_rel = df_TEST_pred_rf_summary['Area_diff'].mean() / df_TEST_pred_rf_summary['Area'].mean() * 100
Area_pred_mean_error_TEST, Area_pred_mean_error_TEST_rel

(-0.5550705831232754, -0.023095434811554332)

In [213]:
#MAE for Area and Area_pred

mae_area_pred_TEST = mean_absolute_error(df_TEST_pred_rf_summary['Area'],df_TEST_pred_rf_summary['Area_pred'])
mae_area_pred_TEST

22.29609452309002

In [214]:
#RMSE for Area and Area_pred

rmse_area_pred_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_rf_summary['Area'],df_TEST_pred_rf_summary['Area_pred'])))
rmse_area_pred_TEST

520.2329417766805

In [215]:
#Looking for min and max values in the df

df_TEST_pred_rf_summary[(df_TEST_pred_rf_summary['Price_diff'] < -7) |
                      (df_TEST_pred_rf_summary['Price_diff'] > 5)]

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
578,75.00,1.13,0.00,180.30,75.00,-179.17,84.75,0.00,84.75,100.00,1
602,0.00,180.30,66.65,1.13,-66.65,179.17,0.00,75.31,-75.31,-inf,1
1327,17.00,38.27,0.00,180.30,17.00,-142.03,650.59,0.00,650.59,100.00,2
1351,0.00,180.30,15.64,50.07,-15.64,130.23,0.00,783.07,-783.07,-inf,2
2071,17.00,39.10,0.00,180.30,17.00,-141.20,664.70,0.00,664.70,100.00,3
2095,0.00,180.30,15.13,55.10,-15.13,125.20,0.00,833.64,-833.64,-inf,3
2815,17.00,39.93,0.00,180.30,17.00,-140.37,678.81,0.00,678.81,100.00,4
2839,0.00,180.30,16.15,47.19,-16.15,133.11,0.00,762.17,-762.17,-inf,4
2840,17.00,39.93,17.00,34.85,0.00,5.08,678.81,592.50,86.31,12.72,4
3559,14.00,40.76,0.00,180.30,14.00,-139.54,570.64,0.00,570.64,100.00,5


In [216]:
df_TEST_pred_rf_summary[(df_TEST_pred_rf_summary['Price_diff'] < -7) |
                      (df_TEST_pred_rf_summary['Price_diff'] > 5)]

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
578,75.00,1.13,0.00,180.30,75.00,-179.17,84.75,0.00,84.75,100.00,1
602,0.00,180.30,66.65,1.13,-66.65,179.17,0.00,75.31,-75.31,-inf,1
1327,17.00,38.27,0.00,180.30,17.00,-142.03,650.59,0.00,650.59,100.00,2
1351,0.00,180.30,15.64,50.07,-15.64,130.23,0.00,783.07,-783.07,-inf,2
2071,17.00,39.10,0.00,180.30,17.00,-141.20,664.70,0.00,664.70,100.00,3
2095,0.00,180.30,15.13,55.10,-15.13,125.20,0.00,833.64,-833.64,-inf,3
2815,17.00,39.93,0.00,180.30,17.00,-140.37,678.81,0.00,678.81,100.00,4
2839,0.00,180.30,16.15,47.19,-16.15,133.11,0.00,762.17,-762.17,-inf,4
2840,17.00,39.93,17.00,34.85,0.00,5.08,678.81,592.50,86.31,12.72,4
3559,14.00,40.76,0.00,180.30,14.00,-139.54,570.64,0.00,570.64,100.00,5


In [217]:
#Looking for maximum price difference
#It does not seems very important, since the error comes in a place where unit will not be operating

df_TEST_pred_rf_summary [df_TEST_pred_rf_summary['Area_diff_rel(%)']== float('-inf')]

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
602,0.00,180.30,66.65,1.13,-66.65,179.17,0.00,75.31,-75.31,-inf,1
1351,0.00,180.30,15.64,50.07,-15.64,130.23,0.00,783.07,-783.07,-inf,2
2095,0.00,180.30,15.13,55.10,-15.13,125.20,0.00,833.64,-833.64,-inf,3
2839,0.00,180.30,16.15,47.19,-16.15,133.11,0.00,762.17,-762.17,-inf,4
3583,0.00,180.30,14.95,43.51,-14.95,136.79,0.00,650.46,-650.46,-inf,5
4328,0.00,180.30,16.30,42.64,-16.30,137.66,0.00,695.03,-695.03,-inf,6
5072,0.00,180.30,15.13,59.03,-15.13,121.27,0.00,893.13,-893.13,-inf,7
5816,0.00,180.30,16.49,48.00,-16.49,132.30,0.00,791.46,-791.46,-inf,8
6560,0.00,180.30,15.47,58.18,-15.47,122.12,0.00,900.00,-900.00,-inf,9
7304,0.00,180.30,13.43,75.12,-13.43,105.18,0.00,"1,008.85","-1,008.85",-inf,10


In [218]:
#Looking for minimmum price difference

df_TEST_pred_rf_summary[(df_TEST_pred_rf_summary['Period'] == 11) &
                      (abs(df_TEST_pred_rf_summary['Price_diff']) >= 1)]

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
7593,17.00,44.57,17.00,51.17,0.00,-6.60,757.69,869.88,-112.19,-14.81,11
7594,17.00,44.57,17.00,50.99,0.00,-6.42,757.69,866.80,-109.11,-14.40,11
7595,17.00,44.57,17.00,51.16,0.00,-6.59,757.69,869.72,-112.03,-14.79,11
7596,17.00,44.57,17.00,50.25,0.00,-5.68,757.69,854.32,-96.63,-12.75,11
7597,17.00,44.57,17.00,49.50,0.00,-4.93,757.69,841.55,-83.86,-11.07,11
...,...,...,...,...,...,...,...,...,...,...,...
8155,17.00,44.48,17.00,47.81,0.00,-3.33,756.16,812.81,-56.65,-7.49,11
8156,17.00,44.48,17.00,47.81,0.00,-3.33,756.16,812.81,-56.65,-7.49,11
8157,17.00,44.48,17.00,47.81,0.00,-3.33,756.16,812.81,-56.65,-7.49,11
8158,17.00,44.48,17.00,47.81,0.00,-3.33,756.16,812.81,-56.65,-7.49,11


#### CURVE AREA METRIC WHEN MAXIMUM PRICE IS NOT CONSIDERED --> NO SÉ SI ES MEJOR BORRAR ESTO

When Price is the maximum (180.3€) the unit is not going to be operated, son these areas should not be cosidered to evaluate the quality of the model.

In [219]:
#NO TENGO CLARO QUE ESTA MÉTRICA TENGA MUCHO SENTIDO

In [220]:
def df_pred_summary_noPmax(df):
    
    df_noPmax = df[df['Price'] != df['Price'].max()]
    
    return df_noPmax

In [221]:
#Looking for maximum price

df_TEST_pred_rf_summary_Pmax = df_TEST_pred_rf_summary[df_TEST_pred_rf_summary['Price'] == df_TEST_pred_rf_summary['Price'].max()]

In [222]:
df_TEST_pred_rf_summary_Pmax[df_TEST_pred_rf_summary_Pmax['dE_diff']>1]

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
8786,394.10,180.30,323.60,180.30,70.50,0.00,"71,056.23","58,345.08","12,711.15",17.89,12
8787,319.10,180.30,274.10,180.30,45.00,0.00,"57,533.73","49,420.23","8,113.50",14.10,12
8791,274.10,180.30,136.35,180.30,137.75,0.00,"49,420.23","24,583.90","24,836.33",50.26,12
8792,139.10,180.30,34.10,180.30,105.00,-0.00,"25,079.73","6,148.23","18,931.50",75.49,12


In [223]:
#df without Pmax Areas

df_TEST_pred_rf_summary_noPmax = df_pred_summary_noPmax(df_TEST_pred_rf_summary)
df_TEST_pred_rf_summary_noPmax.head()

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
3,75.00,1.13,75.00,1.13,0.00,0.00,84.75,84.75,0.00,0.00,1
4,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1
5,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1
6,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1
7,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1


In [224]:
df_TEST_pred_rf_summary_noPmax.describe()

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
count,"5,436.00","5,436.00","5,436.00","5,436.00","5,436.00","5,436.00","5,436.00","5,436.00","5,436.00","5,436.00","5,436.00"
mean,35.48,35.23,35.44,35.55,0.04,-0.32,619.20,618.49,0.71,0.09,5.77
std,51.54,12.98,51.56,14.56,2.02,6.47,167.03,170.21,36.33,5.43,3.23
min,14.00,1.13,0.00,1.13,-70.00,-179.17,84.75,0.00,-186.17,-60.00,1.00
25%,17.00,35.99,17.00,35.99,0.00,-0.27,607.07,611.37,-4.58,-0.73,3.00
50%,17.00,39.10,17.00,38.99,0.00,-0.05,664.70,662.77,-0.89,-0.11,6.00
75%,17.00,42.00,17.00,41.88,0.00,0.13,714.00,710.60,2.25,0.34,9.00
max,190.00,48.54,190.00,180.30,75.00,5.08,825.18,943.86,825.18,100.00,11.00


In [225]:
df_TEST_pred_rf_summary_noPmax['Area_diff'].mean()

0.7092143254221341

In [226]:
Area_pred_noPmax_mean_error_TEST = df_TEST_pred_rf_summary_noPmax['Area_diff'].mean()
Area_pred_noPmax_mean_error_TEST_rel = df_TEST_pred_rf_summary_noPmax['Area_diff'].mean() / df_TEST_pred_rf_summary_noPmax['Area'].mean() * 100
Area_pred_noPmax_mean_error_TEST, Area_pred_noPmax_mean_error_TEST_rel

(0.7092143254221341, 0.11453676034051374)

In [227]:
#MAE for Area and Area_pred without Pmax

mae_area_noPmax_pred_TEST = mean_absolute_error(df_TEST_pred_rf_summary_noPmax['Area'],df_TEST_pred_rf_summary_noPmax['Area_pred'])
mae_area_noPmax_pred_TEST

11.23319033535789

In [228]:
#RMSE for Area and Area_pred without Pmax

rmse_area_noPmax_pred_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_rf_summary_noPmax['Area'],df_TEST_pred_rf_summary_noPmax['Area_pred'])))
rmse_area_noPmax_pred_TEST

36.33541235070892

### RANDOM FOREST AREA METRIC EVALUATION - HYPERPARAMENTERS

In [229]:
df_TEST_pred_rf_grid_summary  = df_pred_summary(y_dE_TEST, y_dE_TEST_pred_rf_grid)

In [230]:
Area_pred_grid_mean_error_TEST = df_TEST_pred_rf_grid_summary['Area_diff'].mean()
Area_pred_grid_mean_error_TEST_rel = df_TEST_pred_rf_grid_summary['Area_diff'].mean() / df_TEST_pred_rf_grid_summary['Area'].mean() * 100
Area_pred_grid_mean_error_TEST, Area_pred_grid_mean_error_TEST_rel

(-1.1348438184748526, -0.04721870015774274)

In [231]:
#MAE for Area and Area_pred

mae_area_pred_grid_TEST = mean_absolute_error(df_TEST_pred_rf_grid_summary['Area'],df_TEST_pred_rf_grid_summary['Area_pred'])
mae_area_pred_grid_TEST

23.559521857049994

In [232]:
#RMSE for Area and Area_pred

rmse_area_pred_grid_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_rf_grid_summary['Area'],df_TEST_pred_rf_grid_summary['Area_pred'])))
rmse_area_pred_grid_TEST

519.9641836809028

#### METRICS FOR RANDOM FOREST WITHOUT PRICE MAX PREDICTIONS

In [233]:
df_TEST_pred_grid_summary_noPmax = df_pred_summary_noPmax(df_TEST_pred_rf_grid_summary)

In [234]:
df_TEST_pred_grid_summary_noPmax['Area_diff'].mean()

0.7670028502711695

In [235]:
Area_pred_grid_noPmax_mean_error_TEST = df_TEST_pred_grid_summary_noPmax['Area_diff'].mean()
Area_pred_grid_noPmax_mean_error_TEST_rel = df_TEST_pred_grid_summary_noPmax['Area_diff'].mean() / df_TEST_pred_grid_summary_noPmax['Area'].mean() * 100
Area_pred_grid_noPmax_mean_error_TEST, Area_pred_grid_noPmax_mean_error_TEST_rel

(0.7670028502711695, 0.12386949684033857)

In [236]:
#MAE for Area and Area_pred without Pmax

mae_area_noPmax_pred_grid_TEST = mean_absolute_error(df_TEST_pred_grid_summary_noPmax['Area'],df_TEST_pred_grid_summary_noPmax['Area_pred'])
mae_area_noPmax_pred_grid_TEST

10.42340719709199

In [237]:
#RMSE for Area and Area_pred without Pmax

rmse_area_noPmax_pred_grid_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_grid_summary_noPmax['Area'],df_TEST_pred_grid_summary_noPmax['Area_pred'])))
rmse_area_noPmax_pred_grid_TEST

36.04201225065036

### XGBOOST AREA METRIC EVALUATION

In [238]:
df_TEST_pred_XGB_summary  = df_pred_summary(y_dE_TEST, y_dE_TEST_pred_XGB)

In [239]:
df_TEST_pred_XGB_summary['Area_diff'].mean()

0.4228363552421333

In [240]:
Area_pred_XGB_mean_error_TEST = df_TEST_pred_XGB_summary['Area_diff'].mean()
Area_pred_XGB_mean_error_TEST_rel = df_TEST_pred_XGB_summary['Area_diff'].mean() / df_TEST_pred_XGB_summary['Area'].mean() * 100
Area_pred_XGB_mean_error_TEST, Area_pred_XGB_mean_error_TEST_rel

(0.4228363552421333, 0.017593419243190345)

In [241]:
#MAE for Area and Area_pred

mae_area_pred_XGB_TEST = mean_absolute_error(df_TEST_pred_XGB_summary['Area'],df_TEST_pred_XGB_summary['Area_pred'])
mae_area_pred_XGB_TEST

22.3683249637693

In [242]:
#RMSE for Area and Area_pred

rmse_area_pred_XGB_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_XGB_summary['Area'],df_TEST_pred_XGB_summary['Area_pred'])))
rmse_area_pred_XGB_TEST

534.6519400234259

#### METRICS FOR XGBOOST WITHOUT PRICE MAX PREDICTIONS

In [243]:
df_TEST_pred_XGB_summary_noPmax = df_pred_summary_noPmax(df_TEST_pred_XGB_summary)
df_TEST_pred_XGB_summary_noPmax

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
3,75.00,1.13,74.89,1.16,0.11,-0.03,84.75,86.73,-1.98,-2.33,1
4,120.00,1.13,119.97,1.13,0.03,0.00,135.60,135.54,0.06,0.05,1
5,120.00,1.13,119.95,1.13,0.05,0.00,135.60,135.51,0.09,0.06,1
6,120.00,1.13,120.04,1.13,-0.04,0.00,135.60,135.62,-0.02,-0.01,1
7,120.00,1.13,119.61,1.13,0.39,0.00,135.60,135.12,0.48,0.35,1
...,...,...,...,...,...,...,...,...,...,...,...
8179,17.00,44.48,17.00,44.65,-0.00,-0.17,756.16,758.98,-2.82,-0.37,11
8180,17.00,44.48,17.00,44.64,-0.00,-0.16,756.16,758.88,-2.72,-0.36,11
8181,17.00,44.48,17.00,44.69,-0.00,-0.21,756.16,759.78,-3.62,-0.48,11
8182,17.00,44.48,17.00,44.62,0.00,-0.14,756.16,758.45,-2.29,-0.30,11


In [244]:
df_TEST_pred_XGB_summary_noPmax['Area_diff'].mean()

2.096463785854096

In [245]:
Area_pred_XGB_noPmax_mean_error_TEST = df_TEST_pred_XGB_summary_noPmax['Area_diff'].mean()
Area_pred_XGB_noPmax_mean_error_TEST_rel = df_TEST_pred_XGB_summary_noPmax['Area_diff'].mean() / df_TEST_pred_XGB_summary_noPmax['Area'].mean() * 100
Area_pred_XGB_noPmax_mean_error_TEST, Area_pred_XGB_noPmax_mean_error_TEST_rel

(2.096463785854096, 0.3385749012613538)

In [246]:
#MAE for Area and Area_pred without Pmax

mae_area_noPmax_pred_XGB_TEST = mean_absolute_error(df_TEST_pred_XGB_summary_noPmax['Area'],df_TEST_pred_XGB_summary_noPmax['Area_pred'])
mae_area_noPmax_pred_XGB_TEST

8.676263986105122

In [247]:
#RMSE for Area and Area_pred without Pmax

rmse_area_noPmax_pred_XGB_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_XGB_summary_noPmax['Area'],df_TEST_pred_XGB_summary_noPmax['Area_pred'])))
rmse_area_noPmax_pred_XGB_TEST

35.074766669084894

### XGBOOST - HYPERPARMETERS - AREA METRIC EVALUATION

In [248]:
df_TEST_pred_XGB_grid_summary  = df_pred_summary(y_dE_TEST, y_dE_TEST_pred_chain_XGB_grid)

In [249]:
df_TEST_pred_XGB_grid_summary['Area_diff'].mean()

0.6507031808851427

In [250]:
Area_pred_XGB_grid_mean_error_TEST = df_TEST_pred_XGB_grid_summary['Area_diff'].mean()
Area_pred_XGB_grid_mean_error_TEST_rel = df_TEST_pred_XGB_grid_summary['Area_diff'].mean() / df_TEST_pred_XGB_grid_summary['Area'].mean() * 100
Area_pred_XGB_grid_mean_error_TEST, Area_pred_XGB_grid_mean_error_TEST_rel

(0.6507031808851427, 0.027074525930094618)

In [251]:
#MAE for Area and Area_pred

mae_area_pred_XGB_grid_TEST = mean_absolute_error(df_TEST_pred_XGB_grid_summary['Area'],df_TEST_pred_XGB_grid_summary['Area_pred'])
mae_area_pred_XGB_grid_TEST

22.898237031417942

In [252]:
#RMSE for Area and Area_pred

rmse_area_pred_XGB_grid_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_XGB_grid_summary['Area'],df_TEST_pred_XGB_grid_summary['Area_pred'])))
rmse_area_pred_XGB_grid_TEST

553.9711375238136

#### METRICS FOR XGBOOST WITHOUT PRICE MAX PREDICTIONS

In [253]:
df_TEST_pred_XGB_grid_summary_noPmax = df_pred_summary_noPmax(df_TEST_pred_XGB_grid_summary)
df_TEST_pred_XGB_grid_summary_noPmax

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
3,75.00,1.13,74.91,1.13,0.09,-0.00,84.75,84.65,0.10,0.12,1
4,120.00,1.13,120.21,1.13,-0.21,-0.00,135.60,135.83,-0.23,-0.17,1
5,120.00,1.13,120.01,1.13,-0.01,-0.00,135.60,135.62,-0.02,-0.01,1
6,120.00,1.13,120.01,1.13,-0.01,-0.00,135.60,135.61,-0.01,-0.01,1
7,120.00,1.13,120.04,1.13,-0.04,-0.00,135.60,135.64,-0.04,-0.03,1
...,...,...,...,...,...,...,...,...,...,...,...
8179,17.00,44.48,17.00,45.10,-0.00,-0.62,756.16,766.73,-10.57,-1.40,11
8180,17.00,44.48,17.00,45.06,-0.00,-0.58,756.16,766.08,-9.92,-1.31,11
8181,17.00,44.48,17.00,45.03,-0.00,-0.55,756.16,765.51,-9.35,-1.24,11
8182,17.00,44.48,17.00,45.06,-0.00,-0.58,756.16,766.02,-9.86,-1.30,11


In [254]:
df_TEST_pred_XGB_grid_summary_noPmax['Area_diff'].mean()

0.845374848367716

In [255]:
Area_pred_XGB_grid_noPmax_mean_error_TEST = df_TEST_pred_XGB_grid_summary_noPmax['Area_diff'].mean()
Area_pred_XGB_grid_noPmax_mean_error_TEST_rel = df_TEST_pred_XGB_grid_summary_noPmax['Area_diff'].mean() / df_TEST_pred_XGB_grid_summary_noPmax['Area'].mean() * 100
Area_pred_XGB_grid_noPmax_mean_error_TEST, Area_pred_XGB_noPmax_mean_error_TEST_rel

(0.845374848367716, 0.3385749012613538)

In [256]:
#MAE for Area and Area_pred without Pmax

mae_area_noPmax_pred_XGB_grid_TEST = mean_absolute_error(df_TEST_pred_XGB_grid_summary_noPmax['Area'],df_TEST_pred_XGB_grid_summary_noPmax['Area_pred'])
mae_area_noPmax_pred_XGB_grid_TEST

10.892353768622336

In [257]:
#RMSE for Area and Area_pred without Pmax

rmse_area_noPmax_pred_XGB_grid_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_XGB_grid_summary_noPmax['Area'],df_TEST_pred_XGB_grid_summary_noPmax['Area_pred'])))
rmse_area_noPmax_pred_XGB_grid_TEST

36.23131762392866

### 6. NAIVE MODEL

In this section a dumb model is developed to have a baseline as reference for the future models to be built.

In this case, two dumb models will be created:

* Naive_24h: the target prediction will be the same as is was 24h before
* Naive_mean: the target prediction will be the average of 24h and 168h before

In [258]:
#NAIVE MODEL-24h

y_dE_naive_TEST = []

hours = len(df_X_dE_TEST)

for hour in range(0,hours):
    y_dE_naive_TEST.append(df_X_dE_TEST[hour][['Energy_24h', 'Price_24h']])

In [259]:
df_TEST_pred_naive_summary  = df_pred_summary(y_dE_TEST, y_dE_naive_TEST)

In [260]:
df_TEST_pred_naive_summary.columns

Index(['dE', 'Price', 'dE_pred', 'Price_pred', 'dE_diff', 'Price_diff', 'Area',
       'Area_pred', 'Area_diff', 'Area_diff_rel(%)', 'Period'],
      dtype='object')

In [261]:
#MAE for Area and Area_pred

mae_pred_naive_TEST = mean_absolute_error(df_TEST_pred_naive_summary[['dE','Price']],df_TEST_pred_naive_summary[['dE_pred','Price_pred']])
mae_pred_naive_TEST

0.34119847670250897

In [262]:
#RMSE for Area and Area_pred

rmse_pred_naive_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_naive_summary[['dE','Price']],df_TEST_pred_naive_summary[['dE_pred','Price_pred']])))
rmse_pred_naive_TEST

5.557005754831408

In [263]:
#MAE for Area and Area_pred

mae_area_pred_naive_TEST = mean_absolute_error(df_TEST_pred_naive_summary['Area'],df_TEST_pred_naive_summary['Area_pred'])
mae_area_pred_naive_TEST

19.157890905017922

In [264]:
#RMSE for Area and Area_pred

rmse_area_pred_naive_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_naive_summary['Area'],df_TEST_pred_naive_summary['Area_pred'])))
rmse_area_pred_naive_TEST

519.643875137234

In [265]:
df_TEST_pred_naive_summary_noPmax = df_pred_summary_noPmax(df_TEST_pred_naive_summary)
df_TEST_pred_naive_summary_noPmax

,dE,Price,dE_pred,Price_pred,dE_diff,Price_diff,Area,Area_pred,Area_diff,Area_diff_rel(%),Period
3,75.00,1.13,75.00,1.13,0.00,0.00,84.75,84.75,0.00,0.00,1
4,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1
5,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1
6,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1
7,120.00,1.13,120.00,1.13,0.00,0.00,135.60,135.60,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...
8179,17.00,44.48,17.00,44.48,0.00,0.00,756.16,756.16,0.00,0.00,11
8180,17.00,44.48,17.00,44.48,0.00,0.00,756.16,756.16,0.00,0.00,11
8181,17.00,44.48,17.00,44.48,0.00,0.00,756.16,756.16,0.00,0.00,11
8182,17.00,44.48,17.00,44.48,0.00,0.00,756.16,756.16,0.00,0.00,11


In [266]:
#MAE for Area and Area_pred naive

mae_area_naive_noPmax_TEST = mean_absolute_error(df_TEST_pred_naive_summary_noPmax['Area'],df_TEST_pred_naive_summary_noPmax['Area_pred'])
mae_area_naive_noPmax_TEST

6.251146063281823

In [267]:
#RMSE for Area and Area_pred

rmse_area_naive_noPmax_TEST = (np.sqrt(mean_squared_error(df_TEST_pred_naive_summary_noPmax['Area'],df_TEST_pred_naive_summary_noPmax['Area_pred'])))
rmse_area_naive_noPmax_TEST

34.694590619678856

## NAIVE MODEL EASIEST THAN THE USED ABOVE (it can be used for all approaches -hour, block or total-)

In [268]:
y_dE_naive_TEST_2 = df_set_dE_TEST[['Energy_24h','Price_24h']]

In [269]:
y_dE_TEST_2 = df_set_dE_TEST[['Energy','Price']]

In [270]:
np.sqrt(mean_squared_error(y_dE_TEST_2,y_dE_naive_TEST_2))

5.557005754831409

In [271]:
mean_absolute_error(y_dE_TEST_2,y_dE_naive_TEST_2)

0.34119847670250897

In [272]:
mean_absolute_error(y_dE_TEST_2['Energy']*y_dE_TEST_2['Price'],
                    y_dE_naive_TEST_2['Energy_24h']*y_dE_naive_TEST_2['Price_24h'])

19.157890905017922

In [273]:
np.sqrt(mean_squared_error(y_dE_TEST_2['Energy']*y_dE_TEST_2['Price'],
                           y_dE_naive_TEST_2['Energy_24h']*y_dE_naive_TEST_2['Price_24h']))

519.6438751372341

In [274]:
mean_absolute_error(df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Energy']*
                    df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Price'],
                    df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Energy_24h']*
                    df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Price_24h'])

6.251146063281823

In [275]:
np.sqrt(mean_squared_error(df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Energy']*
                    df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Price'],
                    df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Energy_24h']*
                    df_set_dE_TEST[df_set_dE_TEST['Price']!=df_set_dE_TEST['Price'].max()]['Price_24h']))

34.694590619678856

### MODELS' RESULTS SUMMARY 

In [277]:
models_summary = pd.DataFrame({'MAE': [np.mean(mae_pred_naive_TEST),
                                       np.mean(mae_dE_TEST_rf),
                                       np.mean(mae_dE_TEST_rf_grid),
                                       np.mean(mae_dE_TEST_chain_XGB),
                                       np.mean(mae_dE_TEST_chain_XGB_grid)],
                              'RMSE': [np.mean(rmse_pred_naive_TEST),
                                       np.mean(rmse_dE_TEST_rf),
                                       np.mean(rmse_dE_TEST_rf_grid),
                                       np.mean(rmse_dE_TEST_chain_XGB),
                                       np.mean(rmse_dE_TEST_chain_XGB_grid)],
                              'MAE_area': [mae_area_pred_naive_TEST,
                                           mae_area_pred_TEST,
                                           mae_area_pred_grid_TEST,
                                           mae_area_pred_XGB_TEST,
                                           mae_area_pred_XGB_grid_TEST],
                              'RMSE_area': [rmse_area_pred_naive_TEST,
                                            rmse_area_pred_TEST,
                                            rmse_area_pred_grid_TEST,
                                            rmse_area_pred_XGB_TEST,
                                            rmse_area_pred_XGB_grid_TEST],
                              'MAE_area_noPmax': [mae_area_naive_noPmax_TEST,
                                                  mae_area_noPmax_pred_TEST,
                                                  mae_area_noPmax_pred_grid_TEST,
                                                  mae_area_noPmax_pred_XGB_TEST,
                                                  mae_area_noPmax_pred_XGB_grid_TEST],
                              'RMSE_area_noPmax': [rmse_area_naive_noPmax_TEST,
                                                   rmse_area_noPmax_pred_TEST,
                                                   rmse_area_noPmax_pred_grid_TEST,
                                                   rmse_area_noPmax_pred_XGB_TEST,
                                                   rmse_area_noPmax_pred_XGB_grid_TEST]},
                              index =['Naive', 'Random Forest(basic)', 
                                      'Random Forest(hyper)','XGBosst(basic)','XGBosst(hyper)'])
models_summary

,MAE,RMSE,MAE_area,RMSE_area,MAE_area_noPmax,RMSE_area_noPmax
Naive,0.34,5.56,19.16,519.64,6.25,34.69
Random Forest(basic),0.42,5.34,22.30,520.23,11.23,36.34
Random Forest(hyper),0.42,5.42,23.56,519.96,10.42,36.04
XGBosst(basic),0.40,5.51,22.37,534.65,8.68,35.07
XGBosst(hyper),0.43,5.53,22.90,553.97,10.89,36.23
